# Gradiente descendente

Agora que temos uma compreensão fundamental de derivadas, podemos aplicá-la a um algoritmo amplamente utilizado em otimização e aprendizado de máquina. **O gradiente descendente** nos permite minimizar ou maximizar uma função usando um processo iterativo aproveitando a inclinação de cada variável. Normalmente, no aprendizado de máquina, tentamos minimizar uma função de perda ou maximizar a verossimilhança. Começaremos com um exemplo trivial e, em seguida, o aplicaremos à regressão linear. Também falaremos sobre gradiente descendente estocástico e o que esperar de modelos mais complexos, como o aprendizado profundo.

## Compreendendo a Descida do Gradiente

Imagine que você está em uma cadeia de montanhas à noite, com apenas uma lanterna. Você sabe que, para voltar à cidade em segurança, precisa chegar ao vale, o ponto mais baixo da cadeia de montanhas. Você usa a lanterna para observar a encosta ao seu redor em todas as direções e pisa na direção que mais desce. Você dá passos maiores para encostas maiores e passos menores para encostas menores. Eventualmente, você chegará a um ponto mais baixo na cadeia de montanhas **onde a inclinação é 0**.

img

Agora, considere um caso em que há apenas um vale, ou um mínimo, em toda a cadeia de montanhas. Chamamos isso de problema **convexo** porque há apenas um mínimo. Problemas convexos incluem regressão linear, regressão logística e problemas de programação linear. Estes são bastante simples de resolver.

img

Agora, considere uma paisagem com vários **mínimos locais**, ou múltiplos vales nos quais podemos ficar presos. Como esses vales não revelam uma inclinação que desça mais, é fácil para a descida do gradiente ficar presa neles. Chamamos esses tipos de problemas de **problemas não convexos**, e eles são muito mais difíceis de resolver. Problemas não convexos incluem redes neurais e aprendizado profundo. Normalmente, a descida do gradiente estocástico e outras técnicas aleatórias são usadas para lidar com problemas não convexos, e falaremos sobre isso mais tarde.

img

Metaforicamente, a paisagem é uma função matemática para a qual estamos tentando encontrar o ponto mais baixo.

Vamos aplicar a descida de gradiente a um problema simples.

## Exemplo simples de descida de gradiente

Vamos pegar esta função:

$
f(x) = 3 \left(x + 1\right)^{2} + 1
$ 

Aqui está plotado no SymPy:

In [ ]:
from sympy import * 

x = symbols('x')
f = 3*(x+1)**2 + 1
plot(f, xlim=(-3,1), ylim=(-1,10))

Poderíamos resolver isso algebricamente, primeiro calculando a derivada da função em relação a $ x $ e, então, resolvendo para onde a inclinação é $ 0 $.

In [ ]:
from sympy import * 
from sympy.solvers import solve 

x = symbols('x')
f = 3*(x+1)**2 + 1
dx = diff(f, x)

solve(dx, x)

Embora tenhamos um atalho para resolver este problema simples, não temos atalhos para problemas de aprendizado de máquina mais complexos, nos quais precisamos usar a descida do gradiente. Mas podemos entender a descida do gradiente primeiro aplicando-a a um problema simples como este.

Ainda usaremos a derivada da função, mas iniciaremos um algoritmo de busca em um local aleatório para $ x $ que esteja razoavelmente próximo da solução. Abaixo, inicializaremos um $ x $ aleatório, mas o manteremos no intervalo do nosso gráfico acima para consistência visual. Traçaremos a reta tangente para esse local inicial $ x $.

In [ ]:
import random 

# iniciar x em local aleatório
x_i = random.uniform(-3,1)

# calcular declive em x aleatório
# e reta tangente
m = dx.subs(x, x_i) 
b = -(m * x_i - f.subs(x, x_i))

plot(f, m*x+b, xlim=(-3,1), ylim=(-1,10))

Vamos declarar uma **taxa de aprendizado** de $ 0.05 $, que pega uma fração da inclinação e a subtrai do nosso valor x.

In [ ]:
L = .0001

**Agora execute o bloco de código abaixo várias vezes e observe o que acontece**. Preste atenção ao valor $ x $ e à reta tangente. Para onde ela está convergindo?

In [ ]:
# execute esta célula de código repetidamente
x_i -= m * L 
m = dx.subs(x, x_i) 
b = -(m * x_i - f.subs(x, x_i))
print(f"x = {x_i}")
plot(f, m*x+b, xlim=(-3,1), ylim=(-1,10))

Essa busca por um $ x $ que nos dá uma inclinação de $ 0 $ na reta tangente, e portanto o mínimo, é o que chamamos de gradiente descendente. Vamos reempacotar todo o código acima em um loop `for` que faz isso 1000 vezes. Você verá que a reta converge para o mínimo.

In [ ]:
from sympy import * 
from sympy.solvers import solve 
import random 

# declarar função e derivada
x = symbols('x')
f = 3*(x+1)**2 + 1
dx = diff(f, x) 

# declarar taxa de aprendizagem
L = .01

# iniciar x em local aleatório
x_i = random.uniform(-3,1)

for i in range(1000):
    x_i -= m * L 
    m = dx.subs(x, x_i) 
    b = -(m * x_i - f.subs(x, x_i))

print(f"x = {x_i}")
plot(f, m*x+b, xlim=(-3,1), ylim=(-1,10))

Portanto, no final, a descida do gradiente começa em um local aleatório em nossa função (um $ x $ aleatório) e subtrai repetidamente a inclinação vezes a taxa de aprendizado.

Mas como escolhemos uma taxa de aprendizado?

### Escolhendo uma Taxa de Aprendizado

A **taxa de aprendizado** define o quão agressivamente você deseja que o algoritmo de descida do gradiente se mova em direção ao mínimo. É uma fração da inclinação subtraída do valor $ x $ repetidamente até que a função seja minimizada (se você estiver maximizando a função, some a inclinação vezes a taxa de aprendizado em vez de subtrair).

Quanto maior a taxa de aprendizado, mais rápido o progresso ocorrerá, mas à custa da precisão. Se for muito grande, pode não convergir para o mínimo, pois seria como um gigante pisando no vale repetidamente. Tê-la muito pequena criará mais precisão, mas exigirá mais tempo e passos, como uma formiga descendo no vale. Você precisa encontrar um equilíbrio entre os dois. Experimente a taxa de aprendizagem acima (por exemplo, $ 0,3 $ versus $ 0,001 $) para ver como ela afeta o progresso da descida do gradiente.

## Descida de gradiente multivariável

Vamos analisar esta função multivariável e plotá-la.

$
f(x) = 5 x^{2} + 4 y^{2} + 1
$

In [ ]:
from sympy import * 
from sympy.plotting import plot3d

x, y = symbols('x y')

f = 5*x**2 + 4*y**2 + 1 

plot3d(f)

Novamente, podemos resolver algebricamente o mínimo, mas vamos praticar o uso da descida do gradiente com ela. Como aprendemos na última seção, podemos usar derivadas parciais para encontrar a derivada em relação a cada variável de entrada.

$ 
\Large \frac{\delta}{\delta x} = 10x
$   

$ 
\Large \frac{\delta}{\delta y} = 8y
$

In [ ]:
from sympy import * 
from sympy.plotting import plot3d

x, y = symbols('x y')

f = 5*x**2 + 4*y**2 + 1 
dx = diff(f, x) 
dy = diff(f, y) 

In [ ]:
dx

In [ ]:
dy

Vamos experimentar a descida do gradiente de maneira semelhante à que fizemos anteriormente. Primeiro, declaramos nossas derivadas parciais e uma taxa de aprendizado $ L = 0.05 $. Também começaremos $ x $ e $ y $ em um local aleatório no gráfico acima.

In [ ]:
from sympy import * 
from sympy.plotting import plot3d
import random 

# declarar função e derivada
x,y = symbols('x y')
f = 5*x**2 + 4*y**2 + 1 
dx = diff(f, x) 
dy = diff(f, y)

# declarar taxa de aprendizagem
L = .05

# iniciar x em local aleatório
x_i = random.uniform(-10,10)
y_i = random.uniform(-10,10)

# plot 
dx_i = dx.subs(x, x_i)
dy_i = dy.subs(y, y_i) 
b = -(dx_i * x_i + dy_i * y_i - f.subs([(x, x_i), (y, y_i)]))

plot3d(f, dx_i*x + dy_i * y + b, xlim=(-10,10),ylim=(-10,10))

Execute este bloco de código abaixo repetidamente e você verá o plano linear, capturando a inclinação de $ x $ e $ y $, se deslocando em direção ao mínimo.

In [ ]:
dx_i = dx.subs(x, x_i)
dy_i = dy.subs(y, y_i) 

x_i -= dx_i * L 
y_i -= dy_i * L 

b = -(dx_i * x_i + dy_i * y_i - f.subs([(x, x_i), (y, y_i)]))

print(f"x = {x_i}, y = {y_i}")
plot3d(f, dx_i*x + dy_i * y + b, xlim=(-10,10),ylim=(-10,10))

Podemos reempacotar tudo isso em um único script Python para executar essa descida de gradiente. Você verá que $ x $ e $ y $ convergem muito próximos para $ (0.0) $.

In [ ]:
from sympy import * 
from sympy.plotting import plot3d
import random 

# declarar função e derivada
x,y = symbols('x y')
f = 5*x**2 + 4*y**2 + 1 
dx = diff(f, x) 
dy = diff(f, y)

# declarar taxa de aprendizagem
L = .05

# iniciar x em local aleatório
x_i = random.uniform(-10,10)
y_i = random.uniform(-10,10)

for i in range(1000):
    dx_i = dx.subs(x, x_i)
    dy_i = dy.subs(y, y_i) 

    x_i -= dx_i * L 
    y_i -= dy_i * L 

    b = -(dx_i * x_i + dy_i * y_i - f.subs([(x, x_i), (y, y_i)]))

print(f"x = {x_i}, y = {y_i}")
plot3d(f, dx_i*x + dy_i * y + b, xlim=(-10,10),ylim=(-10,10))

## Gradiente descendente para regressão linear

Vamos aplicar a descida do gradiente a algo um pouco mais próximo da prática do mundo real. Embora a regressão linear tenha técnicas de atalho, como decomposição matricial, é uma boa maneira de entender a descida do gradiente para modelos de aprendizado de máquina baseados em dados. Afinal, redes neurais são compostas por funções lineares dentro de funções não lineares, cujas inclinações e interceptos (ou pesos e vieses) são otimizados com a descida do gradiente. Vamos praticar com uma única função linear.

Uma regressão linear ajusta uma reta (ou plano linear, se houver múltiplas variáveis ​​de entrada) através de alguns dados. A **função de perda** é o que estamos tentando minimizar usando a descida do gradiente, e normalmente será a *soma dos quadrados* ou a *média dos quadrados*. Os **resíduos quadrados** são as diferenças quadradas entre o valor de $ y $ de cada ponto de dados e o valor $ y $ previsto da reta, que, quando somados ou calculados como média, compõem a função de perda.

Aqui estão os resíduos quadrados visualizados abaixo para uma determinada reta e 15 pontos de dados.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Extrair variáveis ​​de entrada (todas as linhas, todas as colunas, exceto a última coluna)
X = np.array([9.8, 8.3, 5.3, 1.3, 3, 0.4, 5.4, 7.3, 3.7, 6.8, 5.6, 2,7.6, 7.9, 1.5])

Y = np.array([8.383017, 7.35061323, 5.31904498, 0.99811892, 2.64478489, 1.12535641,
 5.62574367, 6.82704871, 5.66768037, 6.98267837, 7.23655439, 3.36467504,
 9.82253924, 8.52430761, 1.39760223])

fig, ax = plt.subplots()
ax.set_aspect('equal')


# declarar coeficientes de linha
m, b = 0.9153874397162779, 0.7861238923689651

# plotar quadrados
for x,y in zip(X,Y): 
    residuo = m*x+b - y
    ax.add_patch(Rectangle((x, y), residuo, residuo, alpha=.5, color='orange'))

plt.plot(X, m*X+b)
plt.plot(X, Y, 'o') # gráfico de dispersão
plt.show()

Vamos usar a soma dos quadrados (a soma de todas as áreas quadradas acima) como nossa função de perda.

$
\Large \text{SSE} = \sum_{i=0}^{n} (m x_i + b - y_i)^{2}
$ 

Para ver como é o cenário de perdas, vamos usar o SymPy. Como você pode imaginar, precisamos encontrar os valores de $ m $ e $ b $ que nos levarão ao ponto mais baixo deste gráfico.

In [ ]:
from sympy import *
from sympy.plotting import plot3d
import pandas as pd

m, b, i, n = symbols('m b i n')
x, y = symbols('x y', cls=Function)

soma_dos_quadrados = Sum((m*x(i) + b - y(i)) ** 2, (i, 0, n)) \
    .subs(n, len(X) - 1).doit() \
    .replace(x, lambda i: X[i]) \
    .replace(y, lambda i: Y[i])

plot3d(soma_dos_quadrados)

Encontraremos a derivada da função de perda em relação a $ m $ e em relação a $ b $ usando o SymPy. Observe como podemos suportar múltiplos valores $ x $ e $ y $ especificando `cls=Function` para `symbols()`. Em seguida, usaremos o operador `Sum` para realizar uma soma que totaliza as diferenças quadradas entre os valores $ y $ reais e os valores $ y $ previstos.

In [ ]:
from sympy import *

m, b, i, n = symbols('m b i n')
x, y = symbols('x y', cls=Function)

soma_dos_quadrados = Sum((m*x(i) + b - y(i)) ** 2, (i, 0, n))

d_m = diff(soma_dos_quadrados, m)
d_b = diff(soma_dos_quadrados, b)

In [ ]:
d_m

In [ ]:
d_b

Podemos implementar essas duas derivadas manualmente no NumPy, como mostrado abaixo, e usá-lo para executar a descida do gradiente. Observe como isso se assemelha ao nosso exemplo anterior de descida do gradiente multivariável. Estamos

In [ ]:
import numpy as np 
import pandas as pd
from sympy import *

X = np.array([9.8, 8.3, 5.3, 1.3, 3, 0.4, 5.4, 7.3, 3.7, 6.8, 5.6, 2,7.6, 7.9, 1.5])

Y = np.array([8.383017, 7.35061323, 5.31904498, 0.99811892, 2.64478489, 1.12535641,
 5.62574367, 6.82704871, 5.66768037, 6.98267837, 7.23655439, 3.36467504,
 9.82253924, 8.52430761, 1.39760223])

# Construindo o modelo
m = 1.0
b = 1.0

# A taxa de aprendizagem
L = .001

# O número de iterações
iterations = 100_000

n = float(len(X))  # Número de elementos em X

# Executar Descida de Gradiente
for i in range(iterations):

    # inclinação em relação a m
    D_m = (2 * X * ((m * X + b) - Y)).sum()

    # inclinação em relação a b
    D_b = (2 * ((m * X + b) - Y)).sum()

    # atualiza m e b
    m -= L * D_m
    b -= L * D_b
print(m, b)

Se você quiser continuar usando o SymPy, basta substituir os pontos de dados nas funções derivadas. Pela natureza da implementação de soma no SymPy, você precisará chamar `doit()` e depois `lambdify()` para compilar as funções derivadas com eficiência.

In [ ]:
import numpy as np 
import pandas as pd
from sympy import *

X = np.array([9.8, 8.3, 5.3, 1.3, 3, 0.4, 5.4, 7.3, 3.7, 6.8, 5.6, 2,7.6, 7.9, 1.5])

Y = np.array([8.383017, 7.35061323, 5.31904498, 0.99811892, 2.64478489, 1.12535641,
 5.62574367, 6.82704871, 5.66768037, 6.98267837, 7.23655439, 3.36467504,
 9.82253924, 8.52430761, 1.39760223])


m, b, i, n = symbols('m b i n')
x, y = symbols('x y', cls=Function)

sum_of_squares = Sum((m*x(i) + b - y(i)) ** 2, (i, 0, n))

d_m = diff(sum_of_squares, m) \
    .subs(n, len(X) - 1).doit() \
    .replace(x, lambda i: X[i]) \
    .replace(y, lambda i: Y[i])

d_b = diff(sum_of_squares, b) \
    .subs(n, len(X) - 1).doit() \
    .replace(x, lambda i: X[i]) \
    .replace(y, lambda i: Y[i])

# compilar usando lambdify para computação mais rápida
d_m = lambdify([m, b], d_m)
d_b = lambdify([m, b], d_b)

# Construindo o modelo
m = 0.0
b = 0.0

# A taxa de aprendizagem
L = .001

# O número de iterações
iterations = 100_000

# Executar Descida de Gradiente
for i in range(iterations):

    # atualiza m e b
    m -= d_m(m,b) * L
    b -= d_b(m,b) * L

print("y = {0}x + {1}".format(m, b))

print(m, b)

Agora vamos dar uma olhada no resultado e plotá-lo. Parece muito bom! Essa reta parece se encaixar perfeitamente nos pontos.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

fig, ax = plt.subplots()
ax.set_aspect('equal')

# plotar quadrados
for x,y in zip(X,Y): 
    residual = m*x+b - y
    ax.add_patch(Rectangle((x, y), residual, residual, alpha=.5, color='orange'))

plt.plot(X, m*X+b)
plt.plot(X, Y, 'o') # gráfico de dispersão
plt.show()

## Descida do gradiente estocástico

Seria negligente não mencionar ao menos a **descida do gradiente estocástico**, uma variante da descida do gradiente que amostra aleatoriamente apenas um ou mais pontos de dados de treinamento em cada iteração. Isso ocorre porque percorrer todo o conjunto de dados pode ser computacionalmente custoso para conjuntos de dados maiores e modelos complexos, como o aprendizado profundo. Abaixo, amostramos aleatoriamente apenas um ponto de dados em cada iteração. Você notará que a reta não se ajusta tão agressivamente, e alguma aleatoriedade produzirá valores diferentes de $ m $ e $ b $ a cada vez. Isso provavelmente não é problema, pois outro objetivo é evitar o sobreajuste.

In [ ]:
import numpy as np 
import pandas as pd
import random 
from sympy import *
import matplotlib.pyplot as plt

X = np.array([9.8, 8.3, 5.3, 1.3, 3, 0.4, 5.4, 7.3, 3.7, 6.8, 5.6, 2,7.6, 7.9, 1.5])

Y = np.array([8.383017, 7.35061323, 5.31904498, 0.99811892, 2.64478489, 1.12535641,
 5.62574367, 6.82704871, 5.66768037, 6.98267837, 7.23655439, 3.36467504,
 9.82253924, 8.52430761, 1.39760223])

# Construindo o modelo
m = 1.0
b = 1.0

# A taxa de aprendizagem
L = .001

# O número de iterações
iterations = 100_000

n = float(len(X))  # Número de elementos em X

# Executar Descida de Gradiente
for i in range(iterations):
    j = random.randint(0,len(X)-1)
    _x, _y = X[j], Y[j]
    
    # inclinação em relação a m
    D_m = 2 * _x * ((m * _x + b) - _y)

    # inclinação em relação a b
    D_b = 2 * ((m * _x + b) - _y)

    # atualiza m e b
    m -= L * D_m
    b -= L * D_b
    
print(m, b)

# plotar o resultado
fig, ax = plt.subplots()
ax.set_aspect('equal')

plt.plot(X, m*X+b)
plt.plot(X, Y, 'o') # gráfico de dispersão
plt.show()

## EXERCÍCIO

Abaixo, temos uma função que aceita as variáveis ​​de entrada $ x $ e $ y $.

$ \Large f(x,y) = 3 \left(x + 2\right)^{2} + 0.5 \left(y - 1\right)^{2} $

Encontre os valores de $ x $ e $ y $ que produzem o menor valor nessa função usando o gradiente descendente e, em seguida, plote-o. Preencha o código abaixo substituindo os pontos de interrogação "?" e experimentando com a taxa de aprendizado e as iterações.

In [ ]:
from sympy import * 
from sympy.plotting import plot3d
import random 

# declara função e derivada
x,y = symbols('x y')
f = 3*(x+2)**2 + .5*(y-1)**2
dx = diff(f, x) 
dy = diff(f, y)

# declara taxa de aprendizagem
L = ?

# inicia x em local aleatório
x_i = random.uniform(-10,10)
y_i = random.uniform(-10,10)

for i in range(?):
    dx_i = dx.subs(x, x_i)
    dy_i = dy.subs(y, y_i) 

    x_i -= dx_i * L 
    y_i -= dy_i * L 

    b = -(dx_i * x_i + dy_i * y_i - f.subs([(x, x_i), (y, y_i)]))

# imprime e plota o resultado
print(f"x = {x_i}, y = {y_i}")
plot3d(f, dx_i*x + dy_i * y + b, xlim=(-10,10),ylim=(-10,10))

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

Você deve convergir em $ x = -2 $ e $ y = 1 $. Uma taxa de aprendizado de $ 0,05 $ e $ 1.000 iterações devem ser suficientes.

In [ ]:
from sympy import * 
from sympy.plotting import plot3d
import random 

# declara função e derivada
x,y = symbols('x y')
f = 3*(x+2)**2 + .5*(y-1)**2
dx = diff(f, x) 
dy = diff(f, y)

# declara taxa de aprendizagem 
L = .05

# inicia x em local aleatório
x_i = random.uniform(-10,10)
y_i = random.uniform(-10,10)

for i in range(1000):
    dx_i = dx.subs(x, x_i)
    dy_i = dy.subs(y, y_i) 

    x_i -= dx_i * L 
    y_i -= dy_i * L 

    b = -(dx_i * x_i + dy_i * y_i - f.subs([(x, x_i), (y, y_i)]))

# imprime e plota o resultado
print(f"x = {x_i}, y = {y_i}")
plot3d(f, dx_i*x + dy_i * y + b, xlim=(-10,10),ylim=(-10,10))